In [1]:
import keras
import numpy
import scapy.plist
from scapy.all import *
import networkx as nx
import numpy as np
import scipy.sparse as sp
from scapy.layers.inet import TCP, Ether, IP, UDP
from scapy.layers.dns import DNS
import tensorflow as tf
import tensorflow.keras as tfk
from scipy.linalg import fractional_matrix_power
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
class SgcLayer(tfk.layers.Layer):
    def __init__(self, outputsNumber):
        super(SgcLayer, self).__init__()
        self.outputs = outputsNumber
        pass

    def build(self, input_shape):
        self.teta = self.add_weight("teta",
                                    shape=[input_shape[-1],self.outputs],
                                    trainable=True,
                                    initializer="random_normal")

    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.teta))



In [3]:
class cgnn_model(tf.keras.Model):
    def __init__(self):
        super(cgnn_model, self).__init__()
        self.sgc1 = SgcLayer(516)
        self.sgc2 = SgcLayer(256)
        self.dense = tf.keras.layers.Dense(6)

    def call(self, inputs):
        x = self.sgc1(inputs)
        x = self.sgc2(x)
        x = tf.keras.layers.AveragePooling1D(strides=1, pool_size=(inputs.shape[1]))(x)
        x = self.dense(x)
        x = tf.nn.softmax(x)

        return x

In [4]:

def get_diagonal_degree(A):
    D = np.zeros((len(A), len(A)))
    for i in range(len(A)):
        D[i][i] = np.sum(A[i])
    return D


def sum_matrices(A, B):
    result = np.array([[0 for x in range(len(A))] for y in range(len(A))])
    for i in range(len(A)):
        # iterate through columns
        for j in range(len(A[0])):
            result[i][j] = A[i][j] + B[i][j]
    return np.array(result)


def normalize_D_t(D_t):
    for i in range(len(D_t)):
        D_t[i][i] = 1 / np.sqrt(D_t[i][i])
    return D_t


def normalize_A_t(A_t):
    for i in range(len(A_t)):
        for j in range(len(A_t[0])):
            if A_t[i][j] != 0:
                A_t[i][j] = 1 / np.sqrt(A_t[i][j])
    return A_t


def convertListToArray(Array):
    new_A = np.empty([len(Array), 1500])
    for i in range(len(Array)):
        np.append(new_A, np.array(Array[i]))
    return new_A

def getSX(session):
    X = convertListToArray(session)
    A = calculate_A(session)
    I = np.identity(len(A[0]))
    A_t = sum_matrices(A, I)
    D_t = np.array(get_diagonal_degree(A_t))
    D_t = normalize_D_t(D_t)
    A_t = normalize_A_t(A_t)
    S_temp = np.dot(D_t, A_t)
    S = np.dot(S_temp, D_t)
    X = X.astype(float)
    SX = np.dot(S, X)
    return np.asarray(SX)


def complete(s, l=1500):
    return [i for i in s] + [0] * (l - len(s))


def calculate_A(session):
    X = session
    A = []
    for i in range(len(X)):
        temp = []
        for j in range(len(X)):
            if i == j + 1 or i == j - 1:
                temp.append(1)
            else:
                temp.append(0)
        A.append(temp)
    return np.array(A)



In [5]:
def createGraphFromSession(pcapName):
    file = rdpcap("data/w_hi_chrome/" + pcapName)
    l = []
    for p in file:
        packet_proc = preprocessing(p)
        if packet_proc:
            if complete(packet_proc):
                l.append(complete(packet_proc))
    return getSX(l)

In [6]:
def preprocessing(packet):
    if packet.haslayer(IP):
        packet[IP].src = 0
        packet[IP].dst = 0
        if packet.haslayer(TCP):
            FIN = 0x01
            SYN = 0x02
            ACK = 0x10
            F = packet['TCP'].flags  # this should give you an integer
            if F & FIN or F & SYN or F & ACK or packet.haslayer(DNS):
                if len(packet) <= 66:
                    return None

            w_eth_header = bytes(packet)[14:]
            return w_eth_header
        elif packet.haslayer(UDP):
            w_eth_header = bytes(packet)[14:]
            zero_bytes = bytearray(12)
            new_packet = bytes(w_eth_header[:8]) + zero_bytes + bytes(w_eth_header[8:])
            return new_packet

In [7]:

def getTrainTestGraphs():
    df = pd.read_csv("data/w_hi_chrome/id.csv")
    train_name, test_name, train_label, test_label = train_test_split(df["fname"],
                                                                      df["label"],
                                                                      test_size=0.98,
                                                                      random_state=42)
    GraphsForTrain = []
    GraphsForTest = []

    for i in train_name:
        g = createGraphFromSession(i)

        GraphsForTrain.append(np.ndarray.tolist(g))

    LableForTrain = []
    for i in train_label:
        LableForTrain.append(int(i))

    counter = 0
    for name in df["fname"]:
        print("its here")
        if name not in train_name:
            g = createGraphFromSession(name)

            counter += 1
            GraphsForTest.append(g)
            if counter == 10:
                break

    return GraphsForTrain, list(LableForTrain), GraphsForTest, list(test_label[:10])  # the graphs is SX


In [ ]:
GraphsForTrain, LabelsForTrain, GraphsForTest, LabelsForTest = getTrainTestGraphs()
m = cgnn_model()
m.compile(loss='categorical_crossentropy', optimizer='adam')

max_n = max([len(i) for i in GraphsForTrain])
for g in GraphsForTrain:
    zeros = [0] * 1500
    for i in range(max_n - len(g)):
        g.append(zeros)

graph_to_train = tf.convert_to_tensor(GraphsForTrain,tf.float32)
graph_to_train = tf.cast(graph_to_train, tf.float32)

dict_label = {}
counter = 0
for item in set(LabelsForTrain):
    dict_label[item] = counter
    counter += 1

dict_label[18102.0] = 5 # remove this when i will train on big data!!!!!!!!!!!!!!!!!!!!!!!!!!!!

list_of_lables = []
for item in LabelsForTrain:
    list_temp = [0] * 6
    list_temp[dict_label[item]] = 1
    list_of_lables.append(list_temp)

label_graph_to_train = tf.convert_to_tensor(list_of_lables)


/var/folders/sw/5xpq5b7j45b21f9qt43jm7jr0000gn/T/ipykernel_25305/2498967240.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(session)


In [ ]:
m.fit(graph_to_train, np.reshape(label_graph_to_train, (65, 1, 6)), epochs=30)


In [ ]:
list_of_lables_test = []
for item in LabelsForTest[:10]:
    list_temp = [0] * 6
    list_temp[dict_label[item]] = 1

    list_of_lables_test.append(list_temp)
for g in GraphsForTest[:10]:
    zeros = [0] * 1500
    for i in range(max_n - len(g)):
        np.append(g, zeros)
graph_to_test = tf.convert_to_tensor(GraphsForTrain[:10], tf.float32)
graph_to_test = tf.cast(graph_to_test, tf.float32)

m.evaluate(graph_to_test, np.reshape(list_of_lables_test[:10], (10, 1, 6)))
